In [2]:
'''from google.colab import drive
drive.mount('/content/drive/')
'''

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import pandas as pd
import numpy as np
import json
import keras
import tensorflow as tf
import io
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.layers import Conv1D, Conv2D
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import LabelEncoder
import nltk
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.layers import average
import tensorflow_hub as hub
from keras.layers import Average
from keras.layers import Concatenate
nltk.download('punkt')
from numpy import random
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import gc
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
#plt.switch_backend('agg')
%matplotlib inline

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
root_path='/content/drive/My Drive/offenseval/2020'

In [0]:
file_name = root_path+"/OffenseEval2020Data/task_c_distant.zip"

In [0]:
train_data=pd.read_csv('/content/drive/My Drive/offenseval/2020/task_c_distant_ann.tsv',delimiter='\t')

## Twitter Embeddings

In [11]:
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Sentimix/word2vec_twitter_tokens.bin', binary=True,unicode_errors='ignore')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Cleaning

In [0]:
def remove_pattern(input_txt, pattern,with_space=False):
    r = re.findall(pattern, input_txt)
    if with_space==False:
      for i in r:
        input_txt = re.sub(i, '', input_txt)
    else:
      for i in r:
        input_txt = re.sub(i, ' ', input_txt)
    return input_txt 

import emoji
import pickle
import re
with open('/content/drive/My Drive/Sentimix/helper_data/contractions.pkl','rb')as f:
  contractions=pickle.load(f)


from collections import Counter
contractions=Counter(contractions)
with open('/content/drive/My Drive/Sentimix/helper_data/acronyms.pkl','rb')as f:
  acronyms=pickle.load(f)
acronyms=Counter(acronyms)
def acronym(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if acronyms[word]!=0:
        w_l.append(acronyms[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
with open('/content/drive/My Drive/Sentimix/hinglish_to_english.pickle','rb')as f:
  hing_to_eng=pickle.load(f)
hing_to_eng=Counter(hing_to_eng)
def hindi_se_english(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    w_l=[]
    sent=str(df[column][i])
    for word in sent.split():
      if hing_to_eng[word]!=0:
        w_l.append(hing_to_eng[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
with open('/content/drive/My Drive/Sentimix/Hinglish_utils/Hinglish_Profanity_dict.pkl', 'rb') as handle:
    cuss_dict=pickle.load(handle)
cuss_dict=Counter(cuss_dict)
def replace_cuss(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if cuss_dict[word]!=0:
        w_l.append('abuse')
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
def remove_contraction(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if contractions[word]!=0:
        w_l.append(contractions[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
def remove_pattern_rep(input_txt, pattern,rep_pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
      input_txt = re.sub(i, rep_pattern, input_txt)

    return input_txt 
def cleaning(data_f,cleaning_col,new_col):
  data_f.reset_index(drop=True,inplace=True)
  for i in tqdm(range(data_f.shape[0])):
    data_f[cleaning_col][i]=emoji.demojize(str(data_f[cleaning_col][i]))
  # #data_f[cleaning_col]=replace_cuss(data_f,cleaning_col)
  # data_f[new_col]=np.vectorize(remove_pattern)(data_f[cleaning_col],"_",with_space=True)
  # data_f[new_col]=np.vectorize(remove_pattern)(data_f[new_col],"-",with_space=True)
  # data_f[new_col]=np.vectorize(remove_pattern)(data_f[new_col],":",with_space=True)
  # data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[new_col], "@[\w]*","<USR>")
  # data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[new_col], "http\S+","<URL>")
  data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[cleaning_col], "[0-9]+","<NUM>")
  #data_f[new_col]=hindi_se_english(data_f,cleaning_col)
  data_f[new_col]=remove_contraction(data_f,new_col)
  data_f[new_col]=acronym(data_f,new_col)
  data_f[new_col]=data_f[new_col].str.replace("[^a-zA-Z]<>", " ")
  data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], "~",with_space=False)
  #data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], "!",with_space=True)
  #data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], ".",with_space=True)
  #data_f[new_col] = data_f[new_col].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
  #data_f.drop(['text'],inplace=True)
  return data_f
import numpy as np
#a=cleaning(df_chunky[0],'text','clean_col')


In [15]:
_=cleaning(train_data,'text','clean_col')

  0%|          | 0/188973 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 188973/188973 [00:04<00:00, 42749.55it/s]


In [16]:
train_data.columns

Index(['id', 'text', 'average_ind', 'average_grp', 'average_oth', 'std_ind',
       'std_grp', 'std_oth', 'clean_col'],
      dtype='object')

## Tokenization and Train Test split

In [0]:
df_train,df_test=train_test_split(train_data,test_size=0.2)

In [0]:
max_words =10000
max_len = 25
tok = Tokenizer(max_words)
tok.fit_on_texts(train_data['clean_col'].astype(str))

In [0]:
sequences_train = tok.texts_to_sequences(df_train['clean_col'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [0]:
sequences_dev = tok.texts_to_sequences(df_test['clean_col'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_dev = sequence.pad_sequences(sequences_dev,maxlen=max_len,padding='post',truncating='post')

In [0]:
y_train=df_train[['average_ind','average_grp','average_oth']].to_numpy().argmax(axis=-1)

In [0]:
y_test=df_test[['average_ind','average_grp','average_oth']].to_numpy().argmax(axis=-1)

In [0]:

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})

In [0]:
from keras.utils import to_categorical
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [69]:
print(y_train.shape)
print(y_test.shape)

(151178,)
(37795,)


## Attention

In [0]:
embeddings_index = {}
f = open(os.path.join('/content/drive/My Drive/IR_project/glove.6B', 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
word_index=tok.word_index

In [0]:
embedding_matrix_1 = np.zeros([max_words + 1, 400])
for word, i in tok.word_index.items():
    if word in model:
      if i>max_words:
        break
      embedding_matrix_1[i] = model[word]

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras_self_attention import SeqSelfAttention
from keras.layers import CuDNNGRU,CuDNNLSTM,GlobalMaxPooling1D,GlobalAveragePooling1D
from sklearn.utils import class_weight
class Attention(Layer):
    def __init__(self,step_dim=20,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout

#max_len=

def BidLstm(maxlen, max_features, embed_size):
    inp1 = Input(shape=(maxlen, ))
    #inp2=Input(shape=(1,))
    #x=Embedding(len(word_index)+1,embed_size)(inp1)
    x1 = Embedding(max_words+1,embed_size,weights=[embedding_matrix_1],
                  trainable=True)(inp1)
    # x2 = Embedding(len(tok.word_index) + 1,embed_size_2,weights=[embedding_matrix_2],
    #                trainable=True)(inp1)
    # x3 = Embedding(len(tok.word_index) + 1,embed_size_3,weights=[embedding_matrix_3],
    #                trainable=True)(inp1)
    # x1 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x1)
    # x2 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x2)
    # x3 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x3)   
    # x1 = Attention(maxlen)(x1)
    # x2 = Attention(maxlen)(x2)
    # x3 = Attention(maxlen)(x3)
    # x=  Concatenate()([x1,x2,x3])
    x1 = Bidirectional(CuDNNLSTM(200, return_sequences=True))(x1)   
    x1 = SeqSelfAttention(kernel_regularizer=keras.regularizers.l2(1e-4),
                    bias_regularizer=keras.regularizers.l1(1e-4),
                    attention_regularizer_weight=1e-4,
                    name='Attention')(x1) 
    x2=  GlobalMaxPooling1D()(x1)
    x3= GlobalAveragePooling1D()(x1)
    x=  Concatenate()([x2,x3])
    x = Dropout(0.1)(x)
    #x = Attention(maxlen)(x)
    # layer = Dense(600,name='FC1')(x)
    # layer = Dense(300,activation='relu')(layer)
    layer = Dense(128,activation='relu')(x)
 #   layer = BatchNormalization(name = 'BN1')(layer)
    #layer = Activation('relu')(layer)
    #layer = Dropout(0.4)(layer)
    layer = Dense(64,name='FC2')(layer)
#    layer = BatchNormalization(name = 'BN2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
   # layer=  Concatenate()([layer,inp2])
    # layer=Dense(256,activation='relu')(layer)
    # layer=Dense(128,activation='relu')(layer)
    layer = Dense(3,name='out_layer',activation='softmax')(layer)

    model = Model(inputs=[inp1],outputs=layer)

    return model
model_bi=BidLstm(max_len,max_features=max_words,embed_size=400)

In [0]:
model_bi.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['acc',km.f1_score()])

In [74]:
model_bi.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 25)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 25, 400)      4000400     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 25, 400)      963200      embedding_2[0][0]                
__________________________________________________________________________________________________
Attention (SeqSelfAttention)    (None, 25, 400)      25665       bidirectional_2[0][0]            
____________________________________________________________________________________________

In [0]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))

In [0]:
cp_filepath='/content/drive/My Drive/offenseval/'+'checkpoints/lstm_model_2020b_big_attention.h5'
cp_check_point=keras.callbacks.ModelCheckpoint(cp_filepath, monitor='val_f1_score', verbose=0, save_best_only=True, save_weights_only=False, mode='max', period=1)
es = EarlyStopping(monitor='val_f1_score', mode='max', min_delta=0,patience=2,restore_best_weights=True)
reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [77]:
model_bi.fit([sequences_matrix_train],y_train,validation_data=([sequences_matrix_dev],y_test),epochs=10,batch_size=1024,class_weight=class_weights,callbacks=[es,cp_check_point])


Train on 151178 samples, validate on 37795 samples
Epoch 1/10
151178/151178 [==============================] - 9s 61us/step - loss: 0.5544 - acc: 0.7743 - f1_score: 0.0350 - val_loss: 0.4707 - val_acc: 0.8030 - val_f1_score: 0.0127
Epoch 2/10
151178/151178 [==============================] - 8s 55us/step - loss: 0.3631 - acc: 0.8554 - f1_score: 0.0204 - val_loss: 0.3186 - val_acc: 0.8695 - val_f1_score: 0.0316
Epoch 3/10
151178/151178 [==============================] - 8s 55us/step - loss: 0.3014 - acc: 0.8761 - f1_score: 0.0161 - val_loss: 0.3664 - val_acc: 0.8515 - val_f1_score: 0.0283
Epoch 4/10
151178/151178 [==============================] - 8s 55us/step - loss: 0.2659 - acc: 0.8864 - f1_score: 0.0132 - val_loss: 0.3229 - val_acc: 0.8696 - val_f1_score: 0.0359
Epoch 5/10
151178/151178 [==============================] - 8s 55us/step - loss: 0.2359 - acc: 0.8960 - f1_score: 0.0114 - val_loss: 0.3023 - val_acc: 0.8789 - val_f1_score: 0.0430
Epoch 6/10
151178/151178 [==================

In [0]:
y_preds_dl=model_bi.predict(sequences_matrix_dev)

In [80]:
y_preds_dl

array([[9.82520401e-01, 1.09897470e-02, 6.48984080e-03],
       [9.99997854e-01, 2.12838199e-06, 1.73419661e-08],
       [9.99440491e-01, 4.25356207e-04, 1.34196773e-04],
       ...,
       [9.99999404e-01, 5.71469855e-07, 1.05636069e-08],
       [9.96202528e-01, 3.68329603e-03, 1.14121096e-04],
       [9.99391675e-01, 4.09951987e-04, 1.98411479e-04]], dtype=float32)

In [81]:
from sklearn.metrics import classification_report

#y_pred = model_bi.predict(X_dev, batch_size=30, verbose=1)

print(classification_report(y_test, y_preds_dl.argmax(axis=-1)))

              precision    recall  f1-score   support

           0       0.98      0.89      0.93     30598
           1       0.71      0.86      0.78      4867
           2       0.44      0.78      0.56      2330

    accuracy                           0.88     37795
   macro avg       0.71      0.84      0.76     37795
weighted avg       0.91      0.88      0.89     37795

